## SITCOM Laser Tracker Analysis: TMA_CENTRAL offsets comparison in elevation

In this notebook we look at the TMA_CENTRAL offsets against elevation when measured with respect to the M1M3 surrogate and the M1M3 lower pylons. 

Relevant imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.time import Time

from lsst_efd_client import EfdClient
from lsst.sitcom.vandv.lasertracker import utils

%matplotlib inline
%reload_ext lab_black
%reload_ext autoreload
%autoreload 2

## Retrieve offsets from three different dates

In [ ]:
client = EfdClient('usdf_efd')
fields = ['dX', 'dY', 'dZ', 'dRX', 'dRY', 'target']

# March 10th
start = Time("2024-03-10 18:28:00Z", scale='utc')
end = Time("2024-03-10 19:04:00Z", scale='utc')

offsets_mar09 = await client.select_time_series(
    'lsst.sal.LaserTracker.logevent_offsetsPublish',
    fields,  
    start, 
    end,
)  

# January 4th
start = Time('2024-01-04 08:35:0Z', scale='utc')
end = Time('2024-01-04 09:00:0Z', scale='utc')

offsets_jan04 = await client.select_time_series(
    'lsst.sal.LaserTracker.logevent_offsetsPublish',
    fields,  
    start, 
    end,
)  

# January 6th
start = Time('2024-01-06 09:05:0Z', scale='utc')
end = Time('2024-01-06 09:25:0Z', scale='utc')

offsets_jan06 = await client.select_time_series(
    'lsst.sal.LaserTracker.logevent_offsetsPublish',
    fields,  
    start, 
    end,
)  

In [ ]:
# Extract offsets, target elevation, azimuth and rotator angle by component
data_jan06 = utils.extract_component_data(offsets_jan06, 'TMA_CENTRAL')
data_jan04 = utils.extract_component_data(offsets_jan04, 'TMA_CENTRAL')
data_mar09 = utils.extract_component_data(offsets_mar09, 'TMA_CENTRAL')

## Plot offsets

In [ ]:

plt.figure(figsize = (12, 6))
for idx, field in enumerate(fields[:-1]):
    plt.subplot(2,3,idx + 1)
    plt.plot(data_jan04['target_elevation'], data_jan04[field]- np.mean(data_jan04[field]), '.-', label = 'M1M3 01-04')
    plt.plot(data_jan06['target_elevation'], data_jan06[field]- np.mean(data_jan06[field]), '.-', label = 'M1M3 01-06')
    plt.plot(data_mar09['target_elevation'], data_mar09[field]- np.mean(data_mar09[field]), '.-', label = 'Pylons 03-09')

    # Format plot
    plt.xlabel('Elevation (deg)')
    if idx >= 3:
        plt.ylabel('deg')
    else:
        plt.ylabel('um')
    plt.title(f'TMA_CENTRAL {field} Offsets')
    plt.legend()
    
plt.tight_layout()
plt.savefig('TMACentral_Dependency.png', dpi = 300)